In [ ]:
#The function is for to create a call list where it filters same shift days with caller and the teams.
def call_filter(year, month, day, hour, report, cases):
  import pandas as pd
  import numpy as np
  import datetime

  #Read the case research report
  df_names = pd.read_excel(report)
  #Notices File
  df_cases = pd.read_excel(cases)

  #Left only necessary columns
  df_names = df_names[['Ekip No', 'KKM Protokol','Ekip Sorumlusu', 'Ekipteki Kişiler', 'İhbar/Çağrı Tarihi', 'İhbar/Çağrı  Saati']]
  df_names['Tarih/Saat'] = df_names['İhbar/Çağrı Tarihi'] + df_names['İhbar/Çağrı  Saati']
  df_cases.rename(columns={'Ekip Kodu': 'Ekip No'}, inplace=True)
  merged_df = pd.merge(df_cases, df_names, on=['KKM Protokol', 'Ekip No'], how = 'inner')
  merged_df.drop(columns = ['KKM Seri No', 'Adres', 'Vaka Yeri Açıklaması'], inplace = True)

  # show only where merged_df['Tarih'] starting from year-month-day hour for 24 hours in every 4 days
  start_date = datetime.datetime(year, month, day, hour)
  end_date = start_date + datetime.timedelta(days=1)

  filtered_df = merged_df[
      (merged_df['Tarih'] >= start_date) & (merged_df['Tarih'] < end_date)
  ]

  for i in range(1, 8):
    try:
      start_date += datetime.timedelta(days=4)
      end_date = start_date + datetime.timedelta(days=1)

      temp_df = merged_df[
          (merged_df['Tarih'] >= start_date) & (merged_df['Tarih'] < end_date)
      ]
      filtered_df = pd.concat([filtered_df, temp_df], ignore_index=True)
    except:
      pass
      filtered_df = filtered_df.sort_values(by = ['Ekip Sorumlusu'])[['KKM Protokol', 'Ekip No', 'Tarih', 'Ekip Sorumlusu', 'Ekipteki Kişiler']]
  filtered_df.to_excel('Call List.xlsx')

#Add year-month-firstShiftDay, hour, reports, cases to the function parameters
call_filter(2024, 5, 3, 8, '/content/reports.xlsx', '/content/EŞLEŞEN ONAYSIZ (6).xlsx')